<a href="https://colab.research.google.com/github/Tishya31/Movie_Recommendation_System/blob/main/movie_recommendation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy scikit-learn


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
ratings = pd.read_csv("rating.csv", on_bad_lines='skip', engine='python')
movies = pd.read_csv("movie.csv")

In [5]:
print("Ratings Shape:", ratings.shape)
print("Movies Shape:", movies.shape)
print("Sample Ratings:\n", ratings.head())
print("Sample Movies:\n", movies.head())

Ratings Shape: (65345, 4)
Movies Shape: (27278, 3)
Sample Ratings:
    userId  movieId  rating            timestamp
0       1        2     3.5  2005-04-02 23:53:47
1       1       29     3.5  2005-04-02 23:31:16
2       1       32     3.5  2005-04-02 23:33:39
3       1       47     3.5  2005-04-02 23:32:07
4       1       50     3.5  2005-04-02 23:29:40
Sample Movies:
    movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [6]:
user_movie_matrix = ratings.pivot_table(index="userId", columns="movieId", values="rating")
user_movie_matrix.fillna(0, inplace=True)

In [7]:
similarity_matrix = cosine_similarity(user_movie_matrix)
similarity_df = pd.DataFrame(similarity_matrix, index=user_movie_matrix.index, columns=user_movie_matrix.index)

In [10]:
def recommend_movies(user_id, num_recommendations=5):
      # Get similarity scores for the given user
      similar_users = similarity_df[user_id].sort_values(ascending=False)

      # Get movies not rated by this user
      user_ratings = user_movie_matrix.loc[user_id]
      unrated_movies = user_ratings[user_ratings == 0].index

      scores = {}
      for other_user, similarity in similar_users.items():
          if other_user == user_id:
                      continue
          other_ratings = user_movie_matrix.loc[other_user, unrated_movies]
          for movie, rating in other_ratings.items():
                if rating > 0:
                      if movie not in scores:
                            scores[movie] = 0
                      scores[movie] += similarity * rating

      # Sort movies by score
      sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
      recommended_movie_ids = [movie for movie, score in sorted_scores[:num_recommendations]]

      return movies[movies['movieId'].isin(recommended_movie_ids)]

In [11]:
print("\n Recommended Movies for User 1:")
print(recommend_movies(user_id=1, num_recommendations=5))


 Recommended Movies for User 1:
      movieId                   title                            genres
108       110       Braveheart (1995)                  Action|Drama|War
352       356     Forrest Gump (1994)          Comedy|Drama|Romance|War
476       480    Jurassic Park (1993)  Action|Adventure|Sci-Fi|Thriller
2486     2571      Matrix, The (1999)            Action|Sci-Fi|Thriller
2772     2858  American Beauty (1999)                      Comedy|Drama
